In [22]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np

dat = pd.read_csv("../data/raw/train.csv")
dat1 = pd.read_csv("../data/raw/test.csv")
comunes = [c for c in dat.columns if c in dat1.columns]
numericas = [c for c in comunes if is_numeric_dtype(dat[c])]


In [21]:

datos_train = dat[numericas].describe()
datos_test  = dat1[numericas].describe()
datos_test = datos_test.reindex(datos_train.index)

In [4]:
comparacion = pd.DataFrame(index=datos_train.index)

In [19]:
for col in numericas:
    comparacion[f"{col}_train"] = datos_train[col]
    comparacion[f"{col}_test"] = datos_test[col]

In [20]:
# Calcular diferencias absolutas
for col in numericas:
    comparacion[f"{col}_diff"] = comparacion[f"{col}_train"] - comparacion[f"{col}_test"]

In [18]:
stats_utiles_pct = {'mean', 'std', '25%', '50%', '75%'}
mask_util  = comparacion.index.astype(str).isin(stats_utiles_pct)
stats_menos_utiles = {'count', 'min', 'max'}
mask_menos = comparacion.index.astype(str).isin(stats_menos_utiles)
for col in numericas:
    tr = comparacion[f"{col}_train"]
    te = comparacion[f"{col}_test"].replace(0, np.nan)
    comparacion[f"{col}_diff_pct"] = np.nan
    comparacion.loc[mask_util, f"{col}_diff_pct"] = 100 * (tr[mask_util] - te[mask_util]) / te[mask_util]
    comparacion.loc[mask_menos, f"{col}_diff"] = np.nan
    comparacion.loc[mask_menos, f"{col}_diff_pct"] = np.nan


## redondear resultados 
comparacion = comparacion.round(4)

In [17]:
print("")
print("")
print("------------------------------------------")
print("------------------------------------------")
print("Variables numéricas comparadas:")
print("------------------------------------------")
print("------------------------------------------")
print("")
print("")
print("Variables numéricas comparadas:", numericas)
display(comparacion)
print("")
print("")
print("------------------------------------------")
print("------------------------------------------")




------------------------------------------
------------------------------------------
Variables numéricas comparadas:
------------------------------------------
------------------------------------------


Variables numéricas comparadas: ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


,PassengerId_train,PassengerId_test,Pclass_train,Pclass_test,Age_train,Age_test,SibSp_train,SibSp_test,Parch_train,Parch_test,...,Age_diff,SibSp_diff,Parch_diff,Fare_diff,PassengerId_diff_pct,Pclass_diff_pct,Age_diff_pct,SibSp_diff_pct,Parch_diff_pct,Fare_diff_pct
count,891.0000,418.0000,891.0000,418.0000,714.0000,332.0000,891.0000,418.0000,891.0000,418.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,446.0000,1100.5000,2.3086,2.2656,29.6991,30.2726,0.5230,0.4474,0.3816,0.3923,...,-0.5735,0.0756,-0.0108,-3.4230,-59.4730,1.902,-1.8944,16.9076,-2.7401,-9.6078
std,257.3538,120.8105,0.8361,0.8418,14.5265,14.1812,1.1027,0.8968,0.8061,0.9814,...,0.3453,0.2060,-0.1754,-6.2141,113.0228,-0.685,2.4348,22.9698,-17.8690,-11.1150
min,1.0000,892.0000,1.0000,1.0000,0.4200,0.1700,0.0000,0.0000,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,223.5000,996.2500,2.0000,1.0000,20.1250,21.0000,0.0000,0.0000,0.0000,0.0000,...,-0.8750,0.0000,0.0000,0.0146,-77.5659,100.000,-4.1667,NaN,NaN,0.1849
50%,446.0000,1100.5000,3.0000,3.0000,28.0000,27.0000,0.0000,0.0000,0.0000,0.0000,...,1.0000,0.0000,0.0000,0.0000,-59.4730,0.000,3.7037,NaN,NaN,0.0000
75%,668.5000,1204.7500,3.0000,3.0000,38.0000,39.0000,1.0000,1.0000,0.0000,0.0000,...,-1.0000,0.0000,0.0000,-0.5000,-44.5113,0.000,-2.5641,0.0000,NaN,-1.5873
max,891.0000,1309.0000,3.0000,3.0000,80.0000,76.0000,8.0000,8.0000,6.0000,9.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN




------------------------------------------
------------------------------------------


In [16]:
# tabla de medias
filas_mean = comparacion.loc['mean', :]  # fila 'mean'
resumen_medias = []
for k in numericas:
    tr = filas_mean[f'{k}_train']
    te = filas_mean[f'{k}_test']
    d  = filas_mean.get(f'{k}_diff', np.nan)
    dp = filas_mean.get(f'{k}_diff_pct', np.nan)
    resumen_medias.append([k, tr, te, d, dp])

resumen_medias = pd.DataFrame(resumen_medias, columns=['variable','train_mean','test_mean','delta','delta_%']).sort_values('variable')
resumen_medias = resumen_medias.round({'train_mean':4,'test_mean':4,'delta':4,'delta_%':2})
resumen_medias


,variable,train_mean,test_mean,delta,delta_%
2,Age,29.6991,30.2726,-0.5735,-1.89
5,Fare,32.2042,35.6272,-3.4230,-9.61
4,Parch,0.3816,0.3923,-0.0108,-2.74
0,PassengerId,446.0000,1100.5000,-654.5000,-59.47
1,Pclass,2.3086,2.2656,0.0431,1.90
3,SibSp,0.5230,0.4474,0.0756,16.91
